# Loading packages and colors

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
%run weighted_kde_modified.ipynb
import seaborn as sns
from tqdm import tqdm

from scipy.interpolate import interp1d
import statsmodels.api as sm
from scipy.stats import spearmanr

C:\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


Defining colors

In [2]:
rescale_colors = lambda l: [i/255 for i in l]

In [3]:
blaa_moerk = rescale_colors([0,69,110])
blaa_moerk2 = rescale_colors([64,116,147])
blaa_moerk3 = rescale_colors([191,208,219])

blaa_lys = rescale_colors([127,162,183])
blaa_lys2 = rescale_colors([199,234,251])
blaa_lys3 = rescale_colors([212,239,252])
blaa_lys4 = rescale_colors([226,244,253])

graa_moerk = rescale_colors([83,83,83])
graa_moerk2 = rescale_colors([140,140,140])
graa_lys = rescale_colors([197,197,197])

roed = rescale_colors([160,33,37])
baggrund = rescale_colors([245,250,254])


Load data

In [4]:
data_path = "D:/ipums/00007/"

In [5]:
df = pd.read_csv(data_path+'usa_00007.csv')

# Estimation of densities

The data from IPUMS is weigthed samples. The weights have to enter into the estimation. This simplest way to incorporate this is to simply draw a sample with replacement with numpy, where samples are drawn according to their weights. I draw 100k samples from each year-by-gender distributions.

I remove observations with no income. The magnitude of the results is sensitive to this restriction. The qualitative results however remain unchanged.

In [7]:
year_list = sorted(df[df['year']>=1970]['year'].unique())

In [8]:
n=100000
pdf_d = {}
cdf_d = {}
icdf_d = {}
medians_d = {}
means_d = {}

for y in tqdm(year_list):
    pdf_d[y] = {} 
    cdf_d[y] = {}
    icdf_d[y] = {}
    medians_d[y] = {}
    means_d[y] = {}
    temp = df[(df['year']==y) & (df['inctot']>0)].copy()
    temp['weight_new'] = temp['perwt']/temp['perwt'].sum()
    draw = np.random.choice(temp['inctot'],size=2*n,replace=True,p=temp['weight_new']) #.reshape(-1, 1)
    
    medians_d[y]['all'] = np.median(draw)
    means_d[y]['all'] = np.mean(draw)
    for sex in ['Male','Female']:
              
        temp = df[(df['year']==y) & (df['sex']==sex) & (df['inctot']>0)].copy()
        temp['weight_new'] = temp['perwt']/temp['perwt'].sum()
        draw = np.random.choice(temp['inctot'],size=n,replace=True,p=temp['weight_new']) #.reshape(-1, 1)
        
        medians_d[y][sex] = np.median(draw)
        means_d[y][sex] = np.mean(draw)
                
        dens = sm.nonparametric.KDEUnivariate([float(x) for x in draw])
        
        dens.fit(gridsize=1000)
        pdf_d[y][sex] = interp1d(x=dens.support,y=dens.density)
        cdf_d[y][sex] = interp1d(x=dens.support,y=dens.cdf)
        q_eval = np.linspace(0,1,len(dens.support))
        icdf_d[y][sex] = interp1d(x=q_eval,y=dens.icdf)

100%|█████████████████████████████████████████| 12/12 [37:57<00:00, 196.79s/it]


In [9]:
results_dict = {'pdf':pdf_d,
                'cdf':cdf_d,
                'icdf':icdf_d,
                'medians':medians_d,
                'means':means_d}

Uncomment if you want to save estimated distributions

In [10]:
import pickle
#pickle.dump(results_dict, open( "output/distributions.pkl", "wb" ) )